Another source of potential information on species comes from the IUCN Red List. This notebook makes use of the iucn module and a species Red List search in the bispy package that requires an API key to operate against the IUCN API. It caches discovered information on these species for convenience in use.

# Data Management Considerations
The [IUCN Red List Terms and Conditions](https://www.iucnredlist.org/terms/terms-of-use) are fairly restrictive with regard to reposting, derivative works, and attribution. The API key that must be supplied by the user running our code for access to the IUCN Red List API is obtained by agreeing to adhere to the terms of use. We are currently exploring, with the IUCN team, the best ways that we can leverage these data, comply with the terms of use, and provide value back to the whole community. 

In [1]:
import requests
import bispy
from IPython.display import display
from joblib import Parallel, delayed
from collections import Counter
import jsonschema

iucn = bispy.iucn.Iucn()
bis_utils = bispy.bis.Utils()

import helperfunctions

In [2]:
name_list = helperfunctions.workplan_species()

In [3]:
%%time
# Use joblib to run multiple requests for SGCN records in parallel via scientific names
iucn_results = Parallel(n_jobs=8)(delayed(iucn.search_species)(name, name_source) for name, name_source in name_list)


CPU times: user 1.41 s, sys: 132 ms, total: 1.54 s
Wall time: 1min 10s


In [4]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("../cache/iucn.json", iucn_results))

{'Doc Cache File': 'cache/iucn.json',
 'Number of Documents in Cache': 375,
 'Document Number 162': {'processing_metadata': {'status': 'success',
   'date_processed': '2019-07-23T18:33:54.705227',
   'api': 'http://apiv3.iucnredlist.org/api/v3/species/Notropis ozarcanus',
   'status_message': 'Species Name Matched'},
  'parameters': {'Scientific Name': 'Notropis ozarcanus',
   'Name Source': 'Lookup Name'},
  'iucn_species_summary': {'iucn_taxonid': 202315,
   'iucn_status_code': 'NT',
   'iucn_status_name': 'Near Threatened',
   'record_date': '2012-07-30',
   'iucn_population_trend': 'Decreasing',
   'citation_string': 'NatureServe 2014. Notropis ozarcanus. The IUCN Red List of Threatened Species 2014: e.T202315A19034813. http://dx.doi.org/10.2305/IUCN.UK.2014-3.RLTS.T202315A19034813.en .Downloaded on 23 July 2019',
   'iucn_secondary_identifier': '19034813',
   'resolvable_identifier': 'https://www.iucnredlist.org/species/202315/19034813',
   'doi': 'http://dx.doi.org/10.2305/IUCN.U

# Schema Validation
We are building documentation and data validation details into JSON Schema notation in a set of companion files with each dataset. The following codeblock grabs and displays the schema for this dataset and validates the data we produced and cached to show any errors. The resuilts from running this code should show only the JSON Schema doc for reference. The validate function will only show an output if there is anything to report.

In [26]:
iucn_schema = helperfunctions.load_schema('iucn')
display(iucn_schema)

jsonschema.validate(iucn_results, iucn_schema)

{'definitions': {'items': {'$id': '#items',
   'type': ['object', 'array'],
   'title': 'Generic container for items in a dataset',
   'description': 'A JSON array or object property containing one or more items in a dataset or data structure within a dataset.'}},
 '$schema': 'http://json-schema.org/draft-07/schema#',
 '$id': 'http://data.usgs.gov/property_registry/',
 'type': ['object', 'array'],
 'title': 'Dataset of species summary records from the IUCN Red List',
 'description': 'A dataset containing summary records pulled from querying the IUCN Red List API. Items can indicate a negative search result. Contains a mix of higher level standard properties and custom properties. Data were assembled using a search function build into the experimental bispy software package (https://github.com/usgs-bcb/bispy).',
 'default': None,
 'items': {'$ref': '#/definitions/items',
  'properties': {'processing_metadata': {'$ref': 'common_properties.json#/definitions/processing_metadata'},
   'para

# Result Summary
Perhaps the most interesting current use of the IUCN data is in the area of examining the IUCN Red List categories. The following code blocks report on the number of work plan species found and not found in the IUCN Red List, a listing of the numbers of species in each IUCN Red List category, and a report on where the names came from (original names vs ITIS names).

In [5]:
# "success" means we found the species in the IUCN Red List and "failure" means we did not
Counter(spp["processing_metadata"]["status"] for spp in iucn_results)

Counter({'success': 186, 'failure': 189})

In [6]:
iucn_results_success = [r for r in iucn_results if r["processing_metadata"]["status"] == "success"]

In [7]:
Counter(spp["iucn_species_summary"]["iucn_status_name"] for spp in iucn_results_success)

Counter({'Near Threatened': 37,
         'Vulnerable': 58,
         'Least Concern': 17,
         'Critically Endangered': 7,
         'Data Deficient': 27,
         'Endangered': 31,
         'Near Threatened (in review)': 9})

In [9]:
Counter(spp["parameters"]["Name Source"] for spp in iucn_results_success)

Counter({'Lookup Name': 182, 'Valid ITIS Scientific Name': 4})